IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\M4\NO_FAULT4


In [3]:
guasto=0

VOLO m4 NO FAULT

In [4]:
rcou_m4_fault10 = pd.read_csv("RCOU.csv")
rcou_m4_fault10 = rcou_m4_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault10 = rcou_m4_fault10[((rcou_m4_fault10['C9'] >= potenza) & (rcou_m4_fault10['C10'] >= potenza) & (rcou_m4_fault10['C11'] >= potenza) & (rcou_m4_fault10['C12'] >= potenza) & (rcou_m4_fault10['C13'] >= potenza) & (rcou_m4_fault10['C14']>= potenza))]
rcou_m4_fault10=rcou_m4_fault10.reset_index(drop=True)
display(rcou_m4_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,76367829,1406,1351,1366,1393,1388,1370
1,76399898,1372,1427,1439,1358,1409,1391
2,76420523,1389,1437,1444,1382,1417,1411
3,76439487,1471,1375,1402,1448,1446,1403
4,76471372,1411,1475,1464,1423,1428,1459
...,...,...,...,...,...,...,...
5094,136813708,1657,1369,1584,1468,1512,1545
5095,136823888,1633,1404,1581,1472,1479,1575
5096,136833858,1603,1445,1592,1458,1455,1594
5097,136848913,1631,1407,1600,1450,1499,1558


In [5]:
min=rcou_m4_fault10['TimeUS'][0]
max=rcou_m4_fault10['TimeUS'][len(rcou_m4_fault10)-1]
print(max)
print(min)

136861088
76367829


In [6]:
xkf1_m4_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault10 = xkf1_m4_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault10 = xkf1_m4_fault10[((xkf1_m4_fault10['TimeUS'] >= min) & (xkf1_m4_fault10['TimeUS'] <= max	))]
xkf1_m4_fault10 = xkf1_m4_fault10.reset_index(drop=True)
print(xkf1_m4_fault10)

         TimeUS  Roll  Pitch     Yaw
0      76375630  1.69  -1.08   16.30
1      76386278  1.67  -1.10   16.29
2      76397328  1.62  -1.13   16.29
3      76407963  1.54  -1.13   16.28
4      76418382  1.51  -1.14   16.29
...         ...   ...    ...     ...
5474  136811131 -1.72   0.04  355.34
5475  136821892 -1.71   0.04  355.34
5476  136831987 -1.74   0.05  355.34
5477  136846510 -1.83   0.11  355.33
5478  136859050 -1.73   0.04  355.36

[5479 rows x 4 columns]


In [7]:
att_m4_fault10 = pd.read_csv("ATT.csv")
att_m4_fault10 = att_m4_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault10 = att_m4_fault10[((att_m4_fault10['TimeUS'] >= min) & (att_m4_fault10['TimeUS'] <= max	))]
att_m4_fault10=att_m4_fault10.reset_index(drop=True)
print(att_m4_fault10)

         TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      76377698     0.18  1.69      0.14  -1.08   16.23   16.30
1      76388276     0.18  1.67      0.15  -1.10   16.23   16.29
2      76399032     0.17  1.62      0.15  -1.13   16.23   16.29
3      76409551     0.17  1.54      0.16  -1.13   16.23   16.28
4      76419805     0.17  1.51      0.16  -1.14   16.23   16.29
...         ...      ...   ...       ...    ...     ...     ...
5475  136812901     0.75 -1.72     -1.40   0.04  354.13  355.35
5476  136823075     0.79 -1.71     -1.40   0.04  354.13  355.35
5477  136833108     0.84 -1.74     -1.41   0.05  354.13  355.35
5478  136848053     0.87 -1.83     -1.42   0.11  354.13  355.34
5479  136860307     0.90 -1.73     -1.44   0.04  354.13  355.37

[5480 rows x 7 columns]


In [8]:
from numpy import empty

if len(att_m4_fault10)>len(xkf1_m4_fault10):
    to_add=att_m4_fault10[len(xkf1_m4_fault10):]
    att_m4_fault10=att_m4_fault10[:len(xkf1_m4_fault10)]

if len(xkf1_m4_fault10)>len(att_m4_fault10):
    to_add=xkf1_m4_fault10[len(att_m4_fault10):]

for idx,i in enumerate(att_m4_fault10['Roll']):
    if(xkf1_m4_fault10['Roll'][idx] != i):
        att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Pitch']):
    if(xkf1_m4_fault10['Pitch'][idx] != i):
        att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Yaw']):
    if(xkf1_m4_fault10['Yaw'][idx] != i):
        att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault10=pd.concat([att_m4_fault10,to_add])

print(att_m4_fault10)

C:\Users\mgale\AppData\Local\Temp\ipykernel_6068\3465845594.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_6068\3465845594.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)
C:\Users\mgale\AppData\Local\Temp\ipykernel_6068\3465845594.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pa

         TimeUS  DesRoll   Roll  DesPitch  Pitch  DesYaw      Yaw
0      76377698     0.18  1.690      0.14 -1.080   16.23   16.300
1      76388276     0.18  1.670      0.15 -1.100   16.23   16.290
2      76399032     0.17  1.620      0.15 -1.130   16.23   16.290
3      76409551     0.17  1.540      0.16 -1.130   16.23   16.280
4      76419805     0.17  1.510      0.16 -1.140   16.23   16.290
...         ...      ...    ...       ...    ...     ...      ...
5475  136812901     0.75 -1.715     -1.40  0.040  354.13  355.345
5476  136823075     0.79 -1.725     -1.40  0.045  354.13  355.345
5477  136833108     0.84 -1.785     -1.41  0.080  354.13  355.340
5478  136848053     0.87 -1.780     -1.42  0.075  354.13  355.350
5479  136860307     0.90 -1.730     -1.44  0.040  354.13  355.370

[5480 rows x 7 columns]


In [9]:
#elettricita' che scorre nel motore considerato, al momento ne consideriamo solo 1
esc_0_m4_fault10 = pd.read_csv("ESC_0.csv")
esc_0_m4_fault10 = esc_0_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_0_m4_fault10 = esc_0_m4_fault10[((esc_0_m4_fault10['TimeUS'] >= min) & (esc_0_m4_fault10['TimeUS'] <= max))]
esc_0_m4_fault10=esc_0_m4_fault10.rename(columns={"RPM": "RPM(ESC_0)", "Curr": "Curr(ESC_0)"})

esc_1_m4_fault10 = pd.read_csv("ESC_1.csv")
esc_1_m4_fault10 = esc_1_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_1_m4_fault10 = esc_1_m4_fault10[((esc_1_m4_fault10['TimeUS'] >= min) & (esc_1_m4_fault10['TimeUS'] <= max))]
esc_1_m4_fault10=esc_1_m4_fault10.rename(columns={"RPM": "RPM(ESC_1)", "Curr": "Curr(ESC_1)"})

esc_2_m4_fault10 = pd.read_csv("ESC_2.csv")
esc_2_m4_fault10 = esc_2_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_2_m4_fault10 = esc_2_m4_fault10[((esc_2_m4_fault10['TimeUS'] >= min) & (esc_2_m4_fault10['TimeUS'] <= max))]
esc_2_m4_fault10=esc_2_m4_fault10.rename(columns={"RPM": "RPM(ESC_2)", "Curr": "Curr(ESC_2)"})

esc_3_m4_fault10 = pd.read_csv("ESC_3.csv")
esc_3_m4_fault10 = esc_3_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_3_m4_fault10 = esc_3_m4_fault10[((esc_3_m4_fault10['TimeUS'] >= min) & (esc_3_m4_fault10['TimeUS'] <= max))]
esc_3_m4_fault10=esc_3_m4_fault10.rename(columns={"RPM": "RPM(ESC_3)", "Curr": "Curr(ESC_3)"})

esc_4_m4_fault10 = pd.read_csv("ESC_4.csv")
esc_4_m4_fault10 = esc_4_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_4_m4_fault10 = esc_4_m4_fault10[((esc_4_m4_fault10['TimeUS'] >= min) & (esc_4_m4_fault10['TimeUS'] <= max))]
esc_4_m4_fault10=esc_4_m4_fault10.rename(columns={"RPM": "RPM(ESC_4)", "Curr": "Curr(ESC_4)"})

esc_5_m4_fault10 = pd.read_csv("ESC_5.csv")
esc_5_m4_fault10 = esc_5_m4_fault10.drop(['LineNo','Instance', 'RawRPM', 'Volt', 'Temp', 'CTot', 'MotTemp', 'Err'], axis=1)
esc_5_m4_fault10 = esc_5_m4_fault10[((esc_5_m4_fault10['TimeUS'] >= min) & (esc_5_m4_fault10['TimeUS'] <= max))]
esc_5_m4_fault10=esc_5_m4_fault10.rename(columns={"RPM": "RPM(ESC_5)", "Curr": "Curr(ESC_5)"})

In [10]:
#valori di accelerazione e del giroscopio
imu_0_m4_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault10 = imu_0_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault10 = imu_0_m4_fault10[((imu_0_m4_fault10['TimeUS'] >= min) & (imu_0_m4_fault10['TimeUS'] <= max))]

imu_1_m4_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault10 = imu_1_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault10 = imu_1_m4_fault10[((imu_1_m4_fault10['TimeUS'] >= min) & (imu_1_m4_fault10['TimeUS'] <= max))]

imu_2_m4_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault10 = imu_2_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault10 = imu_2_m4_fault10[((imu_2_m4_fault10['TimeUS'] >= min) & (imu_2_m4_fault10['TimeUS'] <= max))]

imu_m4_fault10 = pd.concat((imu_0_m4_fault10, imu_1_m4_fault10, imu_2_m4_fault10))
imu_m4_fault10=imu_m4_fault10.groupby(imu_m4_fault10.TimeUS, as_index=False).mean()

display(imu_m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,76369982,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346
1,76372073,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593
2,76374333,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186
3,76377446,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467
4,76380512,0.025879,0.046393,0.026676,-0.255138,-0.149288,-9.542381
...,...,...,...,...,...,...,...
20834,136835711,0.067217,-0.045699,-0.004597,-0.369982,0.197985,-9.371787
20835,136841561,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052
20836,136847878,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913
20837,136853859,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997


In [11]:
#sincronizzazione dei tempi tra att e imu_2
m4_fault10 = pd.merge_ordered(imu_m4_fault10,att_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
#sincronizzazione dei tempi tra la tabella ottenuta precedentemente e esc_0 (corrente che passa nel motore)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_0_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_1_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_2_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_3_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_4_m4_fault10)
m4_fault10 = pd.merge_ordered(m4_fault10, esc_5_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
m4_fault10 = pd.merge_ordered(m4_fault10, rcou_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")

In [12]:
m4_fault10["TimeUS"] = m4_fault10["TimeUS"] - m4_fault10.iloc[0]["TimeUS"]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"].astype(int)
m4_fault10["Guasto"] = guasto

In [13]:
last_cell = m4_fault10.iloc[-1, m4_fault10.columns.get_loc('TimeUS')]
print(last_cell)

60493259


In [14]:
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)

In [15]:
df = pd.DataFrame(to_be, columns = ['TimeUS'])

print(df)

         TimeUS
0             0
1          2857
2          5714
3          8571
4         11428
...         ...
21169  60479833
21170  60482690
21171  60485547
21172  60488404
21173  60491261

[21174 rows x 1 columns]


In [16]:
m4_fault10_final = pd.merge_ordered(m4_fault10,df)
m4_fault10_final=m4_fault10_final.fillna(method="ffill").fillna(method="bfill")
print(m4_fault10_final)

         TimeUS      GyrX      GyrY      GyrZ      AccX      AccY       AccZ  \
0             0 -0.200508 -0.064201 -0.024340 -0.141876 -0.301092  -9.360346   
1          2153 -0.200508 -0.064201 -0.024340 -0.141876 -0.301092  -9.360346   
2          2857 -0.200508 -0.064201 -0.024340 -0.141876 -0.301092  -9.360346   
3          4244 -0.176018 -0.085531 -0.030017 -0.164276 -0.126333  -9.419593   
4          5714 -0.176018 -0.085531 -0.030017 -0.164276 -0.126333  -9.419593   
...         ...       ...       ...       ...       ...       ...        ...   
55607  60488404 -0.044895  0.002809  0.021365  0.192493  1.075484 -10.305997   
55608  60491261 -0.044895  0.002809  0.021365  0.192493  1.075484 -10.305997   
55609  60492180  0.258792 -0.173701  0.026603  0.690928  1.419903 -10.860137   
55610  60492478  0.258792 -0.173701  0.026603  0.690928  1.419903 -10.860137   
55611  60493259  0.258792 -0.173701  0.026603  0.690928  1.419903 -10.860137   

       DesRoll  Roll  DesPitch  ...  Cu

In [17]:
m4_fault10_final = m4_fault10_final[m4_fault10_final.TimeUS.isin(to_be)]

In [18]:
m4_fault10_final

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,0,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
2,2857,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
4,5714,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
6,8571,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
9,11428,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55599,60479833,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052,0.84,-1.785,-1.41,...,1.75,5839.0,3.95,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0.0
55603,60482690,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
55604,60485547,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
55607,60488404,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0


In [19]:
from datetime import timedelta
m4_fault10_final=m4_fault10_final.reset_index(drop=True)
m4_fault10_final['TimeUS'] = pd.to_datetime(m4_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,...,Curr(ESC_4),RPM(ESC_5),Curr(ESC_5),C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
1,00:00:00.002857,-0.200508,-0.064201,-0.024340,-0.141876,-0.301092,-9.360346,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
2,00:00:00.005714,-0.176018,-0.085531,-0.030017,-0.164276,-0.126333,-9.419593,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
3,00:00:00.008571,0.026623,-0.026614,-0.010593,-0.314321,0.048481,-9.672186,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
4,00:00:00.011428,0.168383,0.042279,0.012229,-0.332445,0.063955,-9.758467,0.18,1.690,0.14,...,1.78,3042.0,2.33,1406.0,1351.0,1366.0,1393.0,1388.0,1370.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21169,00:01:00.479833,-0.131265,0.172866,0.017041,-0.259921,0.339389,-9.677052,0.84,-1.785,-1.41,...,1.75,5839.0,3.95,1603.0,1445.0,1592.0,1458.0,1455.0,1594.0,0.0
21170,00:01:00.482690,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
21171,00:01:00.485547,0.088613,-0.040721,-0.016431,-0.189151,0.475869,-10.086913,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0
21172,00:01:00.488404,-0.044895,0.002809,0.021365,0.192493,1.075484,-10.305997,0.87,-1.780,-1.42,...,1.75,5839.0,3.95,1631.0,1407.0,1600.0,1450.0,1499.0,1558.0,0.0


## Analisi nel tempo e in frequenza

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

## Funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza

In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault10_final)/350)):
        V1=m4_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        #for k in range(len(peaks_index)):
        #[print("%4.4f    \t %3.4f" %(freq[peaks_index[i]], properties['peak_heights'][i])) for i in range(len(peaks_index))]
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab=tab.append(val,ignore_index=True)
    return tab

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
rpm_ESC_0=time_freq_feat('RPM(ESC_0)')
curr_ESC_0=time_freq_feat('Curr(ESC_0)')
rpm_ESC_1=time_freq_feat('RPM(ESC_1)')
curr_ESC_1=time_freq_feat('Curr(ESC_1)')
rpm_ESC_2=time_freq_feat('RPM(ESC_2)')
curr_ESC_2=time_freq_feat('Curr(ESC_2)')
rpm_ESC_3=time_freq_feat('RPM(ESC_3)')
curr_ESC_3=time_freq_feat('Curr(ESC_3)')
rpm_ESC_4=time_freq_feat('RPM(ESC_4)')
curr_ESC_4=time_freq_feat('Curr(ESC_4)')
rpm_ESC_5=time_freq_feat('RPM(ESC_5)')
curr_ESC_5=time_freq_feat('Curr(ESC_5)')
merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,rpm_ESC_0,curr_ESC_0,rpm_ESC_1,curr_ESC_1,rpm_ESC_2,curr_ESC_2,rpm_ESC_3,curr_ESC_3,rpm_ESC_4,curr_ESC_4,rpm_ESC_5,curr_ESC_5]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto
display(df_merged)

d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_6068\2624265074.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\mgale\AppData\Local\Temp\ipykernel_6068\2624265074.py:33: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
d:\Program Files\Python311\Lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: 

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),...,amp2(RPM(ESC_5)),<>(Curr(ESC_5)),var(Curr(ESC_5)),kurt(Curr(ESC_5)),RMS(Curr(ESC_5)),freq1(Curr(ESC_5)),freq2(Curr(ESC_5)),amp1(Curr(ESC_5)),amp2(Curr(ESC_5)),Guasto
0,0.0,-0.044203,0.010318,0.334582,0.110645,75.0,0.0,4.446807,4.385723,0.043998,...,15999.734798,3.346866,0.608613,-1.598924,3.436334,120.0,118.0,51.651935,8.193605,0
1,1.0,-0.044190,0.010288,0.343789,0.110508,66.0,0.0,4.601441,4.522906,0.044342,...,15890.215833,3.348778,0.608166,-1.595228,3.438133,125.0,123.0,51.463596,8.378338,0
2,2.0,-0.044136,0.010260,0.351586,0.110359,0.0,72.0,4.673353,4.422251,0.044639,...,15714.579808,3.350680,0.607715,-1.591497,3.439920,120.0,118.0,51.262695,8.548961,0
3,3.0,-0.044138,0.010231,0.361115,0.110229,0.0,71.0,4.803700,4.214711,0.044951,...,15475.677146,3.352571,0.607259,-1.587730,3.441696,123.0,121.0,51.049655,8.705999,0
4,4.0,-0.044310,0.010213,0.367417,0.110215,0.0,70.0,4.872315,4.335054,0.045347,...,15176.554864,3.354451,0.606798,-1.583929,3.443462,120.0,118.0,50.824893,8.849910,0
5,5.0,-0.044457,0.010192,0.374945,0.110180,0.0,82.0,4.948821,4.628848,0.045766,...,14820.426076,3.356320,0.606333,-1.580093,3.445216,125.0,123.0,50.588823,8.981078,0
6,6.0,-0.044430,0.010163,0.383704,0.110040,0.0,71.0,5.086346,4.641571,0.045999,...,14463.115717,3.358067,0.605720,-1.576117,3.446830,122.0,120.0,50.301873,9.060191,0
7,7.0,-0.044458,0.010135,0.393571,0.109924,0.0,72.0,5.203164,4.171603,0.046160,...,14054.056520,3.359804,0.605103,-1.572108,3.448434,123.0,121.0,50.004493,9.128220,0
8,8.0,-0.044740,0.010135,0.392417,0.110040,0.0,74.0,5.227955,4.167863,0.046493,...,13595.813672,3.361532,0.604485,-1.568068,3.450029,122.0,120.0,49.697121,9.185990,0
9,9.0,-0.044739,0.010107,0.401857,0.109912,0.0,73.0,5.353085,4.591012,0.046607,...,13091.075475,3.363250,0.603863,-1.563996,3.451614,123.0,121.0,49.380191,9.234164,0


In [23]:
path_file = path_file.replace(r"M4\NO_FAULT4", "")
os.chdir(path_file)
df_merged.to_csv('m4_nofault.csv', index=False)
print(path_file)

d:\GitHub\ProgettoManutenzionePreventiva\
